In [ ]:
import os,sys,psycopg2,ipywidgets;
from ipywidgets import FileUpload;
from IPython.display import display;
from contextlib import closing;

sys.path.append(os.path.join(os.path.expanduser('~'),'notebooks'));
import common;

dbse = os.environ['POSTGRESQL_DB'];
host = os.environ['POSTGRESQL_HOST'];
port = os.environ['POSTGRESQL_PORT'];
user = 'cipsrv_upload';
pswd = os.environ['POSTGRESQL_CIP_PASS'];


In [ ]:
cs = "dbname=%s user=%s password=%s host=%s port=%s" % (
     dbse
    ,user
    ,pswd
    ,host
    ,port
);

try:
    conn = psycopg2.connect(cs);
    conn.set_session(autocommit=True);
except:
    raise Exception("database connection error");

print("database is ready");


In [ ]:
stem = ipywidgets.Text(
     value       = None
    ,placeholder = 'Enter indexing stem here.'
    ,description = 'Stem:'
    ,disabled    = False
    ,continuous_update=False
);
display(stem);

output = ipywidgets.Output();
display(output);

@output.capture()
def on_value_change(change):
    print('stem: ' + stem.value);
    
stem.observe(on_value_change,names='value');


In [ ]:
upstem = stem.value;
with closing(conn.cursor()) as cursor:
    for item in ['_control','_points','_lines','_areas']:
        cursor.execute("SELECT COUNT(*) FROM cipsrv_upload." + upstem + item + " ");
        result = cursor.fetchone();
        print(upstem + item + ": " + str(result[0]));
    

In [ ]:
%%time

with closing(conn.cursor()) as cursor:
    cursor.execute("""
DO $$
DECLARE
    n INTEGER;
    s VARCHAR;
BEGIN
    CALL cipsrv_engine.cipsrv_batch_index(%(upstem)s,n,s);
    RAISE WARNING '%% %%',n,s;
END;
$$;
    """,{"upstem":upstem});
    for notice in conn.notices:
        print(notice);
        

In [ ]:
conn.close();